In [1]:
import os
import datetime as dt
import pandas as pd
import numpy as np
import time

main_dir = os.getcwd()
os.chdir(main_dir + '\\Leaps Pulling')
cad_dir = main_dir + '\\Historical Queries\\CAD'
usd_dir = main_dir + '\\Historical Queries\\US'
whale_dir = main_dir + '\\Historical Queries\\Whales'

from helpers import *
from yahoo_query import *

os.chdir(main_dir)

In [30]:
def create_dgi(annual_data):
    dgi_data = annual_data[['totalLiab',
                             'totalStockholderEquity',
                             'longTermInvestments',
                             'shortTermInvestments',
                             'netIncome','operatingIncome',
                             'totalRevenue','dividendsPaid',
                             'investments','Underlying']]
    dgi_data['dividendsPaid'] = -dgi_data['dividendsPaid']

    dgi_data['payoutRatio'] = dgi_data.dividendsPaid/dgi_data.netIncome
    dgi_data['debtEquityRatio'] = dgi_data.totalLiab/dgi_data.totalStockholderEquity
    dgi_data['netMargin'] = dgi_data.netIncome/dgi_data.totalRevenue
    dgi_data['roic'] = dgi_data.operatingIncome/(dgi_data.longTermInvestments + dgi_data.shortTermInvestments)

    dgi_data = dgi_data[['Underlying','payoutRatio','debtEquityRatio','netMargin','roic','dividendsPaid']]
    return dgi_data

# DGI Portion

def dgi_scores(annual_df,keystats, min_dgi_score):
    dgi_df = create_dgi(annual_df).fillna(0).replace(-np.inf,0).replace(np.inf,0)

    dgi_df_scores = pd.DataFrame(columns = ['divGrowth','payoutChange','payoutRatio',
                                             'debtEquityRatio','netMargin','roic'],
                                  index = dgi_df.Underlying.drop_duplicates().tolist()).fillna(0)

    for ticker in dgi_df.Underlying.drop_duplicates():
        curr_dgi = dgi_df[dgi_df.Underlying == ticker].sort_index()
        if len(curr_dgi) - 1:
            if sum(curr_dgi['dividendsPaid'].pct_change() >= 0.02) == len(curr_dgi) -1:
                dgi_df_scores.loc[ticker, 'divGrowth'] = 1
            if sum(curr_dgi.payoutRatio.pct_change() <= 1) == len(curr_dgi) - 1:
                dgi_df_scores.loc[ticker, 'payoutChange'] = 1
            if curr_dgi.payoutRatio.tail(3).mean() <= 0.6:
                dgi_df_scores.loc[ticker, 'payoutRatio'] = 1
            if sum(curr_dgi.debtEquityRatio.tail(3) <= 1.5) == len(curr_dgi) - 1:
                dgi_df_scores.loc[ticker, 'debtEquityRatio'] = 1
            if sum(curr_dgi.netMargin.tail(3) >= 0.07) == len(curr_dgi) - 1:
                dgi_df_scores.loc[ticker, 'netMargin'] = 1
            if sum(curr_dgi.roic.tail(3) >= 0.2) == len(curr_dgi) - 1:
                dgi_df_scores.loc[ticker, 'roic'] = 1

    dgi_df_scores['score'] = dgi_df_scores.sum(axis = 1)
    dgi_df_scores = dgi_df_scores[dgi_df_scores.score >= min_dgi_score].join(keystats[['currentPrice','returnOnAssets',
                                                                                       'returnOnEquity','sector']])

    dgi_df_scores = dgi_df_scores.sort_values('returnOnAssets', ascending = False)
    dgi_df_scores['roaRank'] = range(1,len(dgi_df_scores) + 1)
    dgi_df_scores = dgi_df_scores.sort_values('returnOnEquity', ascending = False)
    dgi_df_scores['roeRank'] = range(1,len(dgi_df_scores) + 1)
    dgi_df_scores['roRank'] = dgi_df_scores.roaRank + dgi_df_scores.roeRank
    dgi_df_scores = dgi_df_scores.sort_values('roRank')
    return dgi_df_scores[['currentPrice','score','roRank','sector']]

# filterd_dgi = dgi_df[(dgi_df['debtEquityAvg'] <= 1.5) &
#        (dgi_df['divGrowth'] >= 0.03) &
#        (dgi_df['payoutChange'] <= 0) &
#        (dgi_df['payoutAvg'] <= 0.6) & (dgi_df['payoutAvg'] >= 0) &
#        (dgi_df['netMarginAvg'] >= 0.07) &
#        (dgi_df['roicAvg'] >= 0.2)].sort_values('divGrowth', ascending = False)



# LEAPs Portion

def leaps_scores(data_df, keystats, min_score):
    try:
        df = data_df[['Underlying', 'earnings', 'netIncome','totalRevenue','totalStockholderEquity']]
    except:
        df = data_df[['Underlying', 'epsActual', 'netIncome','totalRevenue','totalStockholderEquity']]
    ks = keystats[['52WeekChange','currentPrice','sector','forwardPE','forwardEps','pegRatio']]
    ks['52WeekChange'] = pd.to_numeric(ks['52WeekChange'].replace('Infinity',0))

    df['profitMargin'] = df.netIncome/df.totalRevenue
    df['roe'] = df.netIncome/df.totalStockholderEquity

    # us_quarterly[['Underlying', 'retainedEarnings', 'netIncome','totalRevenue','totalStockholderEquity']]

    leap_scores = pd.DataFrame(columns = ['earningsGrowth','profitMargins','profitMarginChange',
                                          'roe','roeChange'],
                               index = df.Underlying.drop_duplicates().tolist()).fillna(0)

    for ticker in df.Underlying.drop_duplicates():
        curr_data = df[df.Underlying == ticker].sort_index()
        curr_len = len(curr_data) - 1
        if curr_len > 1:
            try:
                if sum(curr_data.earnings.pct_change() >= 0.05) == curr_len:
                    leap_scores.loc[ticker, 'earningsGrowth'] = 1
            except:
                if sum(curr_data.epsActual.pct_change() > 0) == curr_len:
                    leap_scores.loc[ticker, 'earningsGrowth'] = 1
            if sum(curr_data.profitMargin.tail(3) >= 0.1) == curr_len:
                leap_scores.loc[ticker, 'profitMargins'] = 1
            if sum(curr_data.profitMargin.pct_change() >= 0) == curr_len:
                leap_scores.loc[ticker, 'profitMarginChange'] = 1
            if sum(curr_data.roe.tail(3) >= 0.1) == curr_len:
                leap_scores.loc[ticker, 'roe'] = 1
            if sum(curr_data.roe.pct_change() >= 0) == curr_len:
                leap_scores.loc[ticker, 'roeChange'] = 1

    leap_scores['score'] = leap_scores.sum(axis = 1)
    leap_scores = leap_scores[leap_scores.score >= min_score].join(ks).sort_values('52WeekChange', ascending = False)
    return leap_scores


# DGI Filter

In [31]:
os.chdir(cad_dir)

file_date = '2018-09-19'
cad_annual = pd.read_csv('cad_annual_{}.csv'.format(file_date), index_col = 0)
cad_keystats = pd.read_csv('cad_keystats_{}.csv'.format(file_date), index_col = 0)
cad_quarterly = pd.read_csv('cad_quarterly_{}.csv'.format(file_date), index_col = 0)

os.chdir(main_dir)

os.chdir(usd_dir)

file_date = '2018-09-19'
us_annual = pd.read_csv('us_annual_{}.csv'.format(file_date), index_col = 0)
us_keystats = pd.read_csv('us_keystats_{}.csv'.format(file_date), index_col = 0)
us_quarterly = pd.read_csv('us_quarterly_{}.csv'.format(file_date), index_col = 0)

os.chdir(main_dir)

os.chdir(whale_dir)

file_date = '2018-09-21'
whale_df = pd.read_csv('whales {}.csv'.format(file_date), index_col = 0)

hedgeFunds = whale_df[whale_df.index == 'hedgeFunds'].dropna()
hedgeFunds.index = hedgeFunds.ticker

allFunds = whale_df[whale_df.index == 'allFirms'].dropna()
allFunds.index = allFunds.ticker

os.chdir(main_dir)

In [32]:
cad_dgi_scores = dgi_scores(cad_annual, cad_keystats, 4)
print("CAD")
cad_dgi_scores.head(20).join(cad_keystats[['forwardPE','forwardEps','pegRatio']])

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#index

CAD


,currentPrice,score,roRank,sector,forwardPE,forwardEps,pegRatio
TCN.TO,11.17,4,8,Real Estate,12.839081,0.87,NaN
BAD.TO,27.49,4,10,Industrials,14.243524,1.93,NaN
WTE.TO,25.88,4,11,Basic Materials,14.065217,1.84,NaN
CNR.TO,111.36,5,12,Industrials,17.903538,6.22,2.27
LUC.TO,2.36,4,12,Basic Materials,18.153847,0.13,-11.72
MG.TO,70.33,4,20,Consumer Cyclical,9.364847,7.51,0.71
ENGH.TO,80.38,5,21,Technology,33.491665,2.40,5.71
TIH.TO,64.51,4,21,Industrials,18.431429,3.50,0.96
MRU.TO,40.96,4,22,Consumer Defensive,14.075601,2.91,2.00
RCH.TO,29.93,5,24,Consumer Cyclical,21.378572,1.40,1.56


In [33]:
us_dgi_score = dgi_scores(us_annual, us_keystats, 5)
print("USD")
us_dgi_score.head(20).join(us_keystats[['forwardPE','forwardEps','pegRatio']]).sort_values('roRank')

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#index

USD


,currentPrice,score,roRank,sector,forwardPE,forwardEps,pegRatio
MA,218.120,5,4,Financial Services,29.082666,7.50,1.52
INTU,220.000,5,7,Technology,30.136986,7.30,2.08
TXN,107.110,6,8,Technology,17.331717,6.18,1.27
SBUX,55.470,5,12,Consumer Cyclical,21.011364,2.64,1.65
AMAT,38.980,5,13,Technology,8.8792715,4.39,0.50
ACN,173.105,5,15,Technology,23.778158,7.28,2.52
MKTX,183.210,5,19,Financial Services,35.36873,5.18,1.64
ROL,62.450,5,22,Industrials,51.188522,1.22,6.00
AAPL,217.700,5,24,Technology,15.913742,13.68,1.72
SEIC,62.390,5,24,Financial Services,18.568453,3.36,1.69


In [39]:
us_dgi_picks = us_dgi_score.head(20).join(us_keystats[['forwardPE',
                                        'forwardEps',
                                        'pegRatio']]).sort_values('roRank').join(hedgeFunds[['numOfFirmsHoldingInTop10',
                                                                                             'numOfHolders',
                                                                                             'fundNumPercentChange',
                                                                                             'fundsCreatingNewPos',
                                                                                             'fundsAddingPos',
                                                                                             'fundsClosingPos',
                                                                                             'fundsReducingPos']]).sort_values('fundNumPercentChange', 
                                                                                                                               ascending = False)


us_dgi_picks

,currentPrice,score,roRank,sector,forwardPE,forwardEps,pegRatio,numOfFirmsHoldingInTop10,numOfHolders,fundNumPercentChange,fundsCreatingNewPos,fundsAddingPos,fundsClosingPos,fundsReducingPos
BKE,23.300,5,36,Consumer Cyclical,12.010308,1.94,-1.09,0.0,26.0,30.89,6.0,10.0,5.0,7.0
INTU,220.000,5,7,Technology,30.136986,7.30,2.08,7.0,90.0,22.34,28.0,26.0,5.0,22.0
SEIC,62.390,5,24,Financial Services,18.568453,3.36,1.69,2.0,43.0,15.90,15.0,14.0,5.0,8.0
SWKS,88.790,5,29,Technology,11.267766,7.88,0.93,2.0,67.0,13.67,12.0,19.0,20.0,30.0
BR,132.480,5,35,Industrials,25.925634,5.11,2.20,4.0,61.0,5.85,18.0,18.0,3.0,20.0
AAPL,217.700,5,24,Technology,15.913742,13.68,1.72,91.0,262.0,1.33,23.0,91.0,22.0,117.0
LANC,153.520,5,41,Consumer Defensive,27.862068,5.51,10.37,0.0,18.0,0.96,2.0,6.0,7.0,9.0
SBUX,55.470,5,12,Consumer Cyclical,21.011364,2.64,1.65,5.0,114.0,0.65,23.0,39.0,26.0,42.0
ACN,173.105,5,15,Technology,23.778158,7.28,2.52,8.0,90.0,-1.98,16.0,27.0,26.0,33.0
JKHY,159.570,5,32,Industrials,35.53898,4.49,3.66,2.0,33.0,-2.43,3.0,14.0,9.0,14.0


# LEAPs Filtering

In [35]:
leaps_annual = leaps_scores(us_annual, us_keystats, 4)#.replace('Infinity', 0)
leaps_quarterly = leaps_scores(us_quarterly, us_keystats, 4)#.replace('Infinity', 0)


C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:75: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:76: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#inde

In [40]:
leaps_annual_picks = leaps_annual[(leaps_annual.profitMargins > 0) &
             (leaps_annual.earningsGrowth > 0) &
             (leaps_annual.roe > 0) &
             (leaps_annual.profitMarginChange > 0) &
             (pd.to_numeric(leaps_annual['52WeekChange']) > 0.1)].join(hedgeFunds[['numOfFirmsHoldingInTop10',
                                                                                             'numOfHolders',
                                                                                             'fundNumPercentChange',
                                                                                             'fundsCreatingNewPos',
                                                                                             'fundsAddingPos',
                                                                                             'fundsClosingPos',
                                                                                             'fundsReducingPos']]).sort_values('fundNumPercentChange', 
                                                                                                                               ascending = False)

leaps_annual_picks

,earningsGrowth,profitMargins,profitMarginChange,roe,roeChange,score,52WeekChange,currentPrice,sector,forwardPE,forwardEps,pegRatio,numOfFirmsHoldingInTop10,numOfHolders,fundNumPercentChange,fundsCreatingNewPos,fundsAddingPos,fundsClosingPos,fundsReducingPos
ESNT,1,1,1,1,1,5,0.118047,44.4500,Financial Services,8.75,5.08,0.69,4.0,45.0,29.37,9.0,26.0,14.0,8.0
WAL,1,1,1,1,0,4,0.145382,57.6900,Financial Services,12.514099,4.61,1.56,1.0,49.0,21.78,11.0,20.0,10.0,15.0
SIVB,1,1,1,1,0,4,0.832022,319.7500,Financial Services,15.514313,20.61,1.95,3.0,89.0,20.16,27.0,23.0,4.0,31.0
NTRS,1,1,1,1,0,4,0.172387,105.2200,Financial Services,14.61389,7.20,1.04,2.0,69.0,10.33,9.0,24.0,13.0,25.0
WD,1,1,1,1,1,5,0.134927,54.3100,Financial Services,9.595407,5.66,1.02,0.0,23.0,5.71,7.0,7.0,10.0,7.0
ORLY,1,1,1,1,1,5,0.677985,343.0900,Consumer Cyclical,19.48268,17.61,1.28,6.0,89.0,4.49,20.0,23.0,11.0,37.0
TREX,1,1,1,1,0,4,1.090173,80.5500,Basic Materials,31.220932,2.58,1.53,0.0,21.0,3.17,9.0,4.0,3.0,8.0
ICLR,1,1,1,1,0,4,0.303903,146.0000,Healthcare,21.470587,6.80,2.24,2.0,31.0,0.89,7.0,14.0,6.0,8.0
CACC,1,1,1,1,0,4,0.653024,439.8850,Financial Services,14.896207,29.53,0.81,10.0,28.0,-1.07,3.0,8.0,8.0,14.0
PAYX,1,1,1,1,1,5,0.285173,73.9400,Industrials,24.006495,3.08,3.14,3.0,77.0,-1.33,16.0,24.0,15.0,30.0


In [41]:
leaps_quarterly_picks = leaps_quarterly.join(hedgeFunds[['numOfFirmsHoldingInTop10',
                                 'numOfHolders',
                                 'fundNumPercentChange',
                                 'fundsCreatingNewPos',
                                 'fundsAddingPos',
                                 'fundsClosingPos',
                                 'fundsReducingPos']]).sort_values('fundNumPercentChange', ascending = False)

leaps_quarterly_picks

,earningsGrowth,profitMargins,profitMarginChange,roe,roeChange,score,52WeekChange,currentPrice,sector,forwardPE,forwardEps,pegRatio,numOfFirmsHoldingInTop10,numOfHolders,fundNumPercentChange,fundsCreatingNewPos,fundsAddingPos,fundsClosingPos,fundsReducingPos
STOR,1,1,1,0,1,4,0.106583,27.7400,Real Estate,29.827957,0.93,4.82,1.0,37.0,116.93,13.0,17.0,3.0,6.0
CBSH,1,1,1,0,1,4,0.224727,68.9200,Financial Services,17.18703,4.01,1.91,0.0,21.0,65.39,5.0,6.0,1.0,6.0
CFR,1,1,1,0,1,4,0.204482,109.3400,Financial Services,14.79567,7.39,1.16,4.0,45.0,58.61,16.0,16.0,1.0,8.0
NTRS,1,1,1,0,1,4,0.172387,105.2200,Financial Services,14.61389,7.20,1.04,2.0,69.0,10.33,9.0,24.0,13.0,25.0
OXY,1,1,1,0,1,4,0.262282,79.3800,Energy,14.302702,5.55,0.23,10.0,115.0,6.76,22.0,35.0,4.0,50.0
FIBK,1,1,1,0,1,4,0.272853,45.8500,Financial Services,14.064417,3.26,1.99,0.0,22.0,0.97,3.0,8.0,5.0,9.0
BBT,1,1,1,0,1,4,0.121165,51.0900,Financial Services,11.771889,4.34,1.80,2.0,74.0,-10.62,8.0,23.0,19.0,31.0
HCLP,1,1,1,0,1,4,0.351648,12.1756,Basic Materials,4.701004,2.59,-3.44,1.0,13.0,-14.24,1.0,2.0,3.0,5.0
HUN,1,1,1,0,1,4,0.002121,28.8200,Basic Materials,8.28161,3.48,1.02,3.0,68.0,-22.15,6.0,18.0,12.0,40.0
GPMT,1,1,1,0,1,4,0.017647,18.8650,Real Estate,10.539106,1.79,0.57,0.0,22.0,-23.57,3.0,5.0,8.0,8.0


# Writing to CSV

In [42]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
os.chdir(main_dir + '\\Single Name Pulls')
datenow = dt.datetime.today().strftime("%Y-%m-%d")
filename = 'single_names {}.xlsx'.format(datenow)

writer = pd.ExcelWriter(filename, engine='xlsxwriter')

# Write each dataframe to a different worksheet.
cad_dgi_scores.to_excel(writer, sheet_name='ca_dgi')
us_dgi_score.to_excel(writer, sheet_name='us_dgi')
leaps_annual.to_excel(writer, sheet_name='leaps_a')
leaps_quarterly.to_excel(writer, sheet_name = 'leaps_q')

# Close the Pandas Excel writer and output the Excel file.


picknames = 'name_picks {}.xlsx'.format(datenow)

writer = pd.ExcelWriter(picknames, engine='xlsxwriter')

# Write each dataframe to a different worksheet.
us_dgi_picks.to_excel(writer, sheet_name='us_dgi')
leaps_annual_picks.to_excel(writer, sheet_name='leaps_a')
leaps_quarterly_picks.to_excel(writer, sheet_name = 'leaps_q')


writer.save()
os.chdir(main_dir)